In [ ]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np
import numpy.random as random
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from scikit_weak import *

In [ ]:
X, y = load_iris(return_X_y=True)

In [ ]:
sm = DiscreteRandomSmoother(type='set', epsilon=0.5, p_incl = 1.0, p_err=0.50)
target = sm.fit_transform(X,y)
for t in target:
    print(t)

In [ ]:
rs = RoughSetSelector(search_strategy='approximate', epsilon = 0.1, discrete=True, n_iters=10)
rs.fit(X, target)
print(rs.reducts_)

In [ ]:
rs = RoughSetSelector(search_strategy='brute', epsilon = 0.1, discrete=True)
rs.fit(X, target)
print(rs.reducts_)

In [ ]:
rs = RoughSetSelector(search_strategy='brute', epsilon = 0.1, discrete=False,
                      neighborhood='delta', radius=0.1)
rs.fit(X, target)
print(rs.reducts_)

In [ ]:
rs = RoughSetSelector(search_strategy='brute', epsilon = 0.1, discrete=False,
                      neighborhood='nearest', n_neighbors=10)
rs.fit(X, target)
print(rs.reducts_)

In [ ]:
clf = RRLClassifier(resample=True, random_state=42, n_estimators=100)
clf.fit(X, target)
accuracy_score(y, clf.predict(X))

In [ ]:
clf = WeaklySupervisedKNeighborsClassifier()
clf.fit(X, target)
accuracy_score(y, clf.predict(X))

In [ ]:
clf = DELIN()
clf.fit(X, target)
accuracy_score(y, clf.predict(X))